# ⚡ Energy Wall : Analyse de l'Impact Environnemental de l'IA (2024-2030)
**Auteur :** Arthur - Expert Data & IA
**Contexte :** Brief de Crise - Décembre 2029 (Panne Réseau Européenne)

---

### 🎯 Objectif du Notebook
Ce notebook simule l'évolution de la consommation des infrastructures d'Intelligence Artificielle sur la période 2024-2030.
Il vise à expliquer mathématiquement les causes du **"Mur Énergétique"** atteint en décembre 2029 et à proposer des scénarios de sortie de crise.

La modélisation s'appuie sur :
1.  **Les Scaling Laws** (Lois d'échelle) : Lien entre performance, taille du modèle et coût de calcul.
2.  **L'Effet Rebond (Paradoxe de Jevons)** : Impact de l'efficacité matérielle sur la demande globale.
3.  **Les Limites Physiques** : Capacité du réseau électrique, émissions de CO2 et ressources hydriques (Eau).

### 📚 Veille Technologique & Sources de Données

Pour calibrer notre modèle à $t=0$ (2024), nous nous appuyons sur les rapports de référence suivants :

#### 1. Consommation Électrique (Baseline)
*  **Source :** [Agence Internationale de l'Énergie (IEA) - Electricity 2024 Report](https://www.iea.org)[cite: 56].
* **Donnée :** La consommation mondiale des Data Centers est estimée à **460 TWh** en 2022.
* **Hypothèse :** La part spécifique de l'IA est estimée à ~10% de ce total en 2024 (soit ~46 TWh), mais avec une croissance exponentielle supérieure au reste de l'IT.

#### 2. Efficacité Matérielle (Hardware)
* **Source :** *NVIDIA H100 Architecture Whitepaper* & *Google PaLM Paper*.
* **Donnée :** Performance réelle (~1.4 TFLOPS/Watt en FP16) pondérée par un **MFU** (Model Flops Utilization) de 45%.
* **Dynamique :** Nous intégrons une amélioration de l'efficacité de 15% à 50% par an selon les scénarios (Loi de Koomey).

#### 3. Environnement (Carbone & Eau)
*  **CO2 :** Intensité carbone mondiale moyenne de **436 gCO2/kWh** ([Source: World Bank / IEA](https://climateknowledgeportal.worldbank.org))[cite: 64].
*  **Eau :** Le WUE (Water Usage Effectiveness) moyen industriel est de **1.8 L/kWh** ([Source: The Shift Project](https://theshiftproject.org))[cite: 71].

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Configuration pour un rendu esthétique "Pro"
pio.templates.default = "plotly_white"

print("✅ Bibliothèques chargées et configuration prête.")

In [ ]:
# ==========================================
# 1. PARAMÈTRES ET CONSTANTES (SOURCÉS IEA/NVIDIA/SHIFT PROJECT)
# ==========================================

CURRENT_YEAR = 2024
TARGET_YEAR = 2030
YEARS = np.arange(CURRENT_YEAR, TARGET_YEAR + 1)
N = len(YEARS)

# --- 1. Calibrage Marché (Source: IEA) ---
# Facteur d'ajustement pour aligner le modèle bottom-up sur la réalité (~46 TWh IA en 2024)
GLOBAL_MARKET_SCALE = 65 

# --- 2. Hardware & Infra (Source: NVIDIA/Google) ---
# H100 : ~989 TFLOPS FP16 Peak / 700W TDP
PEAK_FLOPS_PER_WATT_2024 = (989 * 1e12) / 700 
# MFU (Model Flops Utilization) : On utilise 45% de la puce en réel
MFU = 0.45 
REAL_EFFICIENCY_2024 = PEAK_FLOPS_PER_WATT_2024 * MFU
PUE_GLOBAL = 1.25 # Power Usage Effectiveness moyen

# --- 3. Baseline Modèle IA (Unité standard) ---
BASELINE_PARAMS = 1000 # Milliards de paramètres (ex: GPT-4 class)
BASELINE_TOKENS = 15   # Trillions de tokens d'entraînement
BASELINE_REQS = 500    # Millions de requêtes/jour (Proxy usage mondial)
TRAINING_VOL = 10      # Nombre de "gros" entraînements par an par acteur majeur

# --- 4. Environnement (Source: Ember/Shift Project) ---
CARBON_INTENSITY_2024 = 436 # gCO2/kWh (Moyenne Monde)
DECARBONIZATION_RATE = 0.96 # -4% par an (Objectif Net Zero)
WUE_2024 = 1.8 # Litres d'eau par kWh (Refroidissement évaporatif)
WUE_IMPROVEMENT_RATE = 0.90 # -10% par an (Optimisation)

In [ ]:
# ==========================================
# 2. MOTEUR DE CALCUL (SCALING LAWS)
# ==========================================

def calc_energy(params_trend, tokens_trend, req_trend, hw_trend):
    """Calcule la consommation d'énergie annuelle (TWh) selon les tendances."""
    energy = []
    for i in range(len(params_trend)):
        N = params_trend[i] * 1e9
        D = tokens_trend * 1e12 
        
        # A. Entraînement (Training) : Loi de Kaplan (6 * N * D)
        flops_train = 6 * N * D * TRAINING_VOL
        e_train = (flops_train / hw_trend[i]) * PUE_GLOBAL
        
        # B. Inférence (Usage) : 2 * N * Tokens générés
        req_year = req_trend[i] * 1e6 * 365
        flops_infer = req_year * (2 * N * 1000) # Hypothèse : 1000 tokens/req moy.
        e_infer = (flops_infer / hw_trend[i]) * PUE_GLOBAL
        
        # Conversion Joules -> TWh + Facteur d'Échelle Marché
        total = ((e_train + e_infer) / 3.6e15) * GLOBAL_MARKET_SCALE
        energy.append(total)
    return np.array(energy)

# ==========================================
# 3. GÉNÉRATION DES 3 SCÉNARIOS (2024-2030)
# ==========================================

# A. SCÉNARIO NORMAL (Business as Usual)
# La Loi de Moore suit (x1.3 eff), la demande croît (x1.8)
p_norm = BASELINE_PARAMS * (1.5 ** np.arange(N)) 
r_norm = BASELINE_REQS * (1.8 ** np.arange(N))   
h_norm = REAL_EFFICIENCY_2024 * (1.3 ** np.arange(N)) 

# B. SCÉNARIO PESSIMISTE (La Crise de 2029)
# Effet Rebond : Hardware efficace mais usage explosif (x2.2) + Modèles géants
p_pess = BASELINE_PARAMS * (1.8 ** np.arange(N)) 
r_pess = BASELINE_REQS * (2.2 ** np.arange(N))   
h_pess = REAL_EFFICIENCY_2024 * (1.15 ** np.arange(N)) 

# C. SCÉNARIO OPTIMISTE (La Solution)
# Sobriété : Small Models (taille baisse), Edge AI, Rupture Hardware (x1.5 eff)
p_opt = BASELINE_PARAMS * (0.9 ** np.arange(N)) 
r_opt = BASELINE_REQS * (1.5 ** np.arange(N)) 
h_opt = REAL_EFFICIENCY_2024 * (1.5 ** np.arange(N)) 

# --- Calculs Finaux ---
e_norm = calc_energy(p_norm, BASELINE_TOKENS, r_norm, h_norm)
e_pess = calc_energy(p_pess, BASELINE_TOKENS, r_pess, h_pess)
e_opt = calc_energy(p_opt, BASELINE_TOKENS, r_opt, h_opt)

# Impact Environnemental
carb_trend = CARBON_INTENSITY_2024 * (DECARBONIZATION_RATE ** np.arange(N))
wue_trend = WUE_2024 * (WUE_IMPROVEMENT_RATE ** np.arange(N))

df = pd.DataFrame({
    'Year': YEARS,
    'E_Pess': e_pess, 'E_Norm': e_norm, 'E_Opt': e_opt,
    'CO2_Pess': e_pess * carb_trend * 0.001,
    'CO2_Norm': e_norm * carb_trend * 0.001,
    'CO2_Opt': e_opt * carb_trend * 0.001,
    'Wat_Pess': e_pess * wue_trend,
    'Wat_Norm': e_norm * wue_trend,
    'Wat_Opt': e_opt * wue_trend
})

print("✅ Modélisation terminée. Données prêtes pour visualisation.")

In [ ]:
def create_storytelling_plot(title, y_label, col_pess, col_norm, col_opt, 
                             txt_pess, txt_norm, txt_opt, 
                             threshold_val=None, threshold_txt=""):
    """Génère un graphique Plotly interactif avec annotations narratives."""
    fig = go.Figure()

    # 1. Pessimiste (Rouge)
    fig.add_trace(go.Scatter(x=df['Year'], y=df[col_pess], name='Pessimiste (Crise)',
                             line=dict(color='#d62728', width=4)))
    # Annotation
    fig.add_annotation(x=2029, y=df[col_pess].iloc[-2], text=f"<b>{txt_pess}</b>",
                       showarrow=True, arrowhead=1, ax=-60, ay=20, bgcolor="#ffcccc", bordercolor="#d62728")

    # 2. Normal (Orange)
    fig.add_trace(go.Scatter(x=df['Year'], y=df[col_norm], name='Normal (Inertiel)',
                             line=dict(color='#ff7f0e', width=3, dash='dot')))
    # Annotation
    fig.add_annotation(x=2028, y=df[col_norm].iloc[-3], text=f"<b>{txt_norm}</b>",
                       showarrow=True, arrowhead=1, ax=-60, ay=-40, bgcolor="#ffeebb", bordercolor="#ff7f0e")

    # 3. Optimiste (Vert)
    fig.add_trace(go.Scatter(x=df['Year'], y=df[col_opt], name='Optimiste (Cible)',
                             line=dict(color='#2ca02c', width=3)))
    # Annotation
    fig.add_annotation(x=2027, y=df[col_opt].iloc[-4], text=f"<b>{txt_opt}</b>",
                       showarrow=True, arrowhead=1, ax=0, ay=-50, bgcolor="#ccffcc", bordercolor="#2ca02c")

    # Ligne de Seuil (Optionnelle)
    if threshold_val:
        fig.add_hline(y=threshold_val, line_dash="dash", line_color="grey", 
                      annotation_text=threshold_txt, annotation_position="top left")

    # Layout Interactif (Boutons Log/Linéaire)
    fig.update_layout(
        title=f"<b>{title}</b>", xaxis_title="Année", yaxis_title=y_label,
        hovermode="x unified",
        margin=dict(r=150),
        updatemenus=[dict(
            type="buttons", direction="left",
            buttons=list([
                dict(args=[{"yaxis.type": "linear"}], label="Vue Mur (Linéaire)", method="relayout"),
                dict(args=[{"yaxis.type": "log"}], label="Vue Détail (Log)", method="relayout")
            ]),
            pad={"r": 10, "t": 10}, showactive=True, x=0, xanchor="left", y=1.15, yanchor="top"
        )]
    )
    return fig

### 📊 1. L'Impasse Énergétique (Consommation en TWh)

Ce graphique montre le cœur de la crise de 2029.
* **Le Mur (Courbe Rouge) :** Résultat du "Paradoxe de Jevons". L'efficacité des puces a réduit le coût, augmentant massivement la demande (Inférence).
* **La Cible (Courbe Verte) :** Ce qu'on aurait pu atteindre avec des *Small Language Models* (SLM) et du calcul local (Edge).

> **Note :** Utilisez le bouton "Vue Détail (Log)" pour voir le décrochage dès 2025.

In [ ]:
fig1 = create_storytelling_plot(
    title="1. L'Impasse Énergétique (TWh)", y_label="Consommation (TWh)",
    col_pess='E_Pess', col_norm='E_Norm', col_opt='E_Opt',
    txt_pess="PARADOXE DE JEVONS<br>Efficacité < Usage",
    txt_norm="INERTIE<br>Demande > Loi de Moore",
    txt_opt="RUPTURE<br>Small Models & Edge AI",
    threshold_val=460, threshold_txt="Capacité Totale France (460 TWh)"
)
fig1.show()

### ☁️ 2. Le Bilan Carbone (Scope 2 - Émissions Indirectes)

L'explosion de la demande électrique a des conséquences directes sur les émissions de CO2.
* **Saturation du Vert :** La demande (+15%/an) croît plus vite que le déploiement des énergies renouvelables (+5%/an), forçant le recours aux énergies fossiles.
* **Comparaison :** En 2030 (scénario rouge), l'IA émettra plus que l'aviation civile mondiale.

In [ ]:
fig2 = create_storytelling_plot(
    title="2. Le Bilan Carbone (Scope 2)", y_label="Millions de Tonnes CO2eq",
    col_pess='CO2_Pess', col_norm='CO2_Norm', col_opt='CO2_Opt',
    txt_pess="RELANCE FOSSILE<br>Le réseau vert sature",
    txt_norm="TROP LENT<br>Décarb. (-2%) vs IA (+15%)",
    txt_opt="DÉCOUPLAGE<br>IA verte sur réseau vert",
    threshold_val=1000, threshold_txt="Aviation Mondiale (~1000 Mt)"
)
fig2.show()

### 💧 3. L'Impact Hydrique (Eau de Refroidissement)

L'infrastructure physique de l'IA (Data Centers) consomme de l'eau potable pour son refroidissement (Tours évaporatives).
* **Le Conflit :** En période de sécheresse, ces milliards de litres entrent en concurrence directe avec l'agriculture et les besoins humains.
* **La Solution Verte :** Le passage à l'*Immersion Cooling* (circuit fermé) permettrait de réduire cette consommation à quasi-zéro.

In [ ]:
fig3 = create_storytelling_plot(
    title="3. L'Impact Hydrique (Eau)", y_label="Giga-Litres (Milliards de L)",
    col_pess='Wat_Pess', col_norm='Wat_Norm', col_opt='Wat_Opt',
    txt_pess="CONFLIT D'USAGE<br>Refroidissement évaporatif",
    txt_norm="OPTIMISATION<br>WUE baisse (1.8 -> 1.5)",
    txt_opt="CIRCUIT FERMÉ<br>Immersion Cooling (WUE~0)"
)
fig3.show()

### 🚀 Conclusion et Plan d'Action

Pour éviter le scénario rouge ("Le Mur de 2029"), trois actions politiques et techniques sont requises immédiatement :

1.  **Délestage Intelligent (Immédiat) :** Priorisation des IA critiques (Santé/Transport) sur les IA de loisir lors des pics de charge.
2.  **Changement de Modèle (Moyen Terme) :** Basculer du "Tout-Cloud" vers l'**Edge AI** et les **Small Language Models (SLM)**, 100x moins énergivores.
3.  **Rupture Technologique (Long Terme) :** Investissement massif dans le **Calcul Neuromorphique** et la Photonique pour sortir de l'architecture von Neumann classique.